# RAiDER in Python

RAiDER can be called and used from with Python. 

In [1]:
# at the most basic level, can "import RAiDER"
import RAiDER

In [2]:
# we'll need some other basic libraries
import datetime
import pyproj
import xarray
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

In [3]:
# to actually do something, we'll need more
# in order: 

# 1. basic data types and input parameters
from RAiDER.llreader import BoundingBox, StationFile
from RAiDER.losreader import Zenith, Conventional, Raytracing

# 2. Weather model access
from RAiDER.models.hrrr import HRRR
from RAiDER.processWM import prepareWeatherModel

# 3. Delay calculation
from RAiDER.delay import tropo_delay

In [16]:
# First define datetime, and AOI
date_time = datetime.datetime(2018,11,13, 23, 0, 0)

# bounding box is given in SNWE format
aoi = BoundingBox([36.8, 36.85, -76.15, -76.05])

In [17]:
date_time

datetime.datetime(2018, 11, 13, 23, 0)

In [18]:
# For the first example, we'll do a zenith calculation at the weather model grid nodes
weather_model = HRRR()
prepareWeatherModel(weather_model, date_time, ll_bounds=aoi.bounds(), makePlots=True)

Weather model HRRR is available from 2016-07-15 00:00:00-Present
✅ Found ┊ model=hrrr ┊ product=prs ┊ 2018-Nov-13 23:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


/Users/jlmd9g/software/miniconda/envs/RAiDER/lib/python3.10/site-packages/herbie/archive.py:692: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)
Can't create file '/Users/jlmd9g/software/RAiDER-docs/notebooks/RAiDER_tutorial/weather_files/HRRR_2018_11_13_T23_00_00.nc.923a8.idx'
Traceback (most recent call last):
  File "/Users/jlmd9g/software/miniconda/envs/RAiDER/lib/python3.10/site-packages/cfgrib/messages.py", line 273, in itervalues
    yield self.filestream.message_from_file(file, errors=errors)
  File "/Users/jlmd9g/software/miniconda/envs/RAiDER/lib/python3.10/site-packages/cfgrib/messages.py", line 340, in message_from_file
    return Message.from_file(file, offset, **kwargs)
  File "/Users/jlmd9g/software/miniconda/envs/RAiDER/lib/python3.10/site-packages/cfgrib/messages.py", line 104, in from_file
    raise EOFError("End of file: %r" % file)

Number of weather model nodes: 55100
Shape of weather model: (20, 19, 145)
Bounds of the weather model: 5693.85/62693.85/1862479.86/1916479.86 (SNWE)
Weather model: HRRR
Mean value of the wet refractivity: 15.119745
Mean value of the hydrostatic refractivity: 113.316704

======Weather Model class object=====
Weather model time: 2018-11-13 23:00:00
Latitude resolution: 0.02702702702702703
Longitude resolution: 0.02702702702702703
Native projection: +proj=lcc +lat_1=38.5 +lat_2=38.5 +lat_0=38.5 +lon_0=262.5 +x_0=0 +y_0=0 +a=6371229 +b=6371229 +units=m +no_defs +type=crs
ZMIN: -100.0
ZMAX: 15000.0
k1 = 0.776
k2 = 0.233
k3 = 3750.0
Humidity type = q
Class name: hrrr
Dataset: hrrr
A: []
B: []
Number of points in Lon/Lat = 20/19
Total number of grid points (3D): 55100



'/Users/jlmd9g/software/RAiDER-docs/notebooks/RAiDER_tutorial/weather_files/HRRR_2018_11_13_T23_00_00_37N_38N_77W_76W.nc'

You can look at the PDF files generated to see slices of the weather model variables at different heights. 

We can also load the weather model using xarray:

In [8]:
# we can get the name of the weather model file by passing the write directory to the 'out_file' method
ds = xarray.load_dataset(weather_model.out_file('weather_files'))

In [9]:
ds

<xarray.Dataset>
Dimensions:      (x: 19, y: 20, z: 145)
Coordinates:
  * x            (x) float64 1.862e+06 1.865e+06 ... 1.913e+06 1.916e+06
  * y            (y) float64 5.694e+03 8.694e+03 ... 5.969e+04 6.269e+04
  * z            (z) float32 -500.0 -300.0 -200.0 ... 7.458e+04 8.03e+04
Data variables:
    CRS          |S1 b''
    latitude     (z, y, x) float64 -3.403e+38 -3.403e+38 ... -3.403e+38
    longitude    (z, y, x) float64 -3.403e+38 -3.403e+38 ... -3.403e+38
    t            (z, y, x) float32 291.8 291.8 292.0 292.0 ... 208.1 208.0 208.0
    p            (z, y, x) float32 1.013e+05 1.013e+05 ... 5.333e+03 5.334e+03
    e            (z, y, x) float32 2.029e+03 2.037e+03 ... 0.061 0.06111
    wet          (z, y, x) float32 91.0 91.32 91.61 ... 0.005353 0.005363
    hydro        (z, y, x) float32 269.3 269.3 269.2 269.1 ... 19.89 19.89 19.89
    wet_total    (z, y, x) float32 0.315 0.3149 0.3147 0.3146 ... 0.0 0.0 0.0
    hydro_total  (z, y, x) float32 3.525 3.525 3.525 3.525 ... 0.0 0.0 0.0 0.0
Attributes:
    Conventions:   CF-1.6
    datetime:      2018_11_13T12_00_00
    date_created:  2022_12_10T16_01_41
    title:         Weather model data and delay calculations

"wet_total" and "hydro_total" are the zenith delays (ZTD) at the weather model grid nodes. 

We can now run the delay calculation using various input query points

In [10]:
los = Zenith()

In [11]:
# calculate ZTD 
ds,_ = tropo_delay(date_time, weather_model.out_file('weather_files'), aoi, los, height_levels=[0, 100, 500, 1000], cube_spacing_m=5000)

Output SNWE: [36.75, 36.85, -76.15, -76.0]
Output cube spacing: 0.05


In [12]:
ds

<xarray.Dataset>
Dimensions:          (z: 4, y: 3, x: 5)
Coordinates:
  * x                (x) float64 -76.15 -76.1 -76.05 -76.0 -75.95
  * y                (y) float64 36.85 36.8 36.75
  * z                (z) int64 0 100 500 1000
Data variables:
    wet              (z, y, x) float64 0.2754 0.276 0.2765 ... 0.1885 0.1881
    hydro            (z, y, x) float64 3.388 3.388 3.388 ... 3.134 3.134 3.135
    cube_projection  int64 0
Attributes:
    Conventions:     CF-1.7
    title:           RAiDER geo cube
    source:          HRRR_2018_11_13_T12_00_00_37N_38N_77W_76W.nc
    history:         2022-12-11 03:10:02.316715 RAiDER
    description:     RAiDER geo cube - zenith
    reference_time:  2018-11-13 12:00:00

In [13]:
print(ds['wet'].shape)

(4, 3, 5)


In [19]:
los = Raytracing('data/S1A_OPER_AUX_POEORB_OPOD_20181203T120749_V20181112T225942_20181114T005942.EOF', time=date_time)

In [20]:
ds,_ = tropo_delay(date_time, 'weather_files/HRRR_2018_11_13_T12_00_00_37N_38N_77W_76W.nc', aoi, los, height_levels=[0, 100, 500, 1000])

Output SNWE: [36.78, 36.87, -76.17, -76.05]
Output cube spacing: 0.03
Look direction: right
Processing slice 1 / 4: 0
Processing slice 2 / 4: 100
Processing slice 3 / 4: 500
Processing slice 4 / 4: 1000


In [21]:
ds

<xarray.Dataset>
Dimensions:          (z: 4, y: 4, x: 6)
Coordinates:
  * x                (x) float64 -76.17 -76.14 -76.11 -76.08 -76.05 -76.02
  * y                (y) float64 36.87 36.84 36.81 36.78
  * z                (z) int64 0 100 500 1000
Data variables:
    wet              (z, y, x) float64 0.4313 0.4318 0.4333 ... 0.328 0.3279
    hydro            (z, y, x) float64 3.344 3.44 3.523 ... 2.381 2.565 2.692
    cube_projection  int64 0
Attributes:
    Conventions:     CF-1.7
    title:           RAiDER geo cube
    source:          HRRR_2018_11_13_T12_00_00_37N_38N_77W_76W.nc
    history:         2022-12-11 03:36:17.396004 RAiDER
    description:     RAiDER geo cube - slant - raytracing
    reference_time:  2018-11-13 23:00:00

In [24]:
# Look at the output
print(ds['wet'].shape)
print(ds['hydro'].shape)
print(ds['wet'].values.ravel().mean())
print(ds['hydro'].values.ravel().mean())

(4, 4, 6)
(4, 4, 6)
0.39695660854055426
3.0515329152900406
